In [ ]:
from pymongo import MongoClient

# MongoDB connection setup
def initialize_mongodb():
    # Replace with your actual MongoDB connection string
    client = MongoClient('mongodb://localhost:27017/')  
    
    # Create or get the database (replace 'day_planner_db' with your preferred database name)
    db = client['day_planner_db']
    
    # Create or get the tasks collection
    global tasks_collection
    tasks_collection = db['tasks']
    
    # Optionally create indexes for better query performance
    tasks_collection.create_index([('date', 1)])
    tasks_collection.create_index([('priority', 1)])
    tasks_collection.create_index([('deadline', 1)])
    
    return tasks_collection

# Initialize MongoDB connection when starting the application
tasks_collection = initialize_mongodb()

In [ ]:
import os
import re
import math
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
from typing import Dict, Any, Optional

from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content
from agentic_patterns.planning_pattern.react_agent import ReactAgent


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.2-90b-text-preview"
GROQ_CLIENT = Groq()

In [ ]:

# Helper functions for parsing natural language inputs
def parse_datetime(text: str) -> Optional[Dict[str, str]]:
    """Extract date and time from natural language text."""
    # Match common date formats
    date_patterns = [
        r'(\d{1,2}[-/]\d{1,2}[-/]\d{2,4})',  # 8-11-2024, 08/11/2024
        r'(\d{4}[-/]\d{1,2}[-/]\d{1,2})',    # 2024-11-08
        r'(tomorrow|today|next week)'          # Natural language dates
    ]
    
    # Match time formats
    time_patterns = [
        r'(\d{1,2}:\d{2})',           # 15:00, 3:00
        r'(\d{1,2}(?::\d{2})?\s*(?:am|pm))',  # 3:00 PM, 3 PM
    ]
    
    date_str = None
    time_str = None
    
    # Extract date
    for pattern in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            date_str = match.group(1)
            if date_str.lower() == 'tomorrow':
                date_str = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
            elif date_str.lower() == 'today':
                date_str = datetime.now().strftime('%Y-%m-%d')
            break
    
    # Extract time
    for pattern in time_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            time_str = match.group(1)
            # Convert 12-hour format to 24-hour
            if 'pm' in time_str.lower():
                hour = int(re.findall(r'\d+', time_str)[0])
                if hour != 12:
                    hour += 12
                time_str = f"{hour:02d}:00"
            elif 'am' in time_str.lower():
                hour = int(re.findall(r'\d+', time_str)[0])
                if hour == 12:
                    hour = 0
                time_str = f"{hour:02d}:00"
            break
    
    if date_str and time_str:
        # Standardize date format
        if '-' in date_str or '/' in date_str:
            try:
                parsed_date = datetime.strptime(date_str, '%d-%m-%Y')
            except ValueError:
                try:
                    parsed_date = datetime.strptime(date_str, '%Y-%m-%d')
                except ValueError:
                    return None
            date_str = parsed_date.strftime('%Y-%m-%d')
            
        return {
            'date': date_str,
            'time': time_str
        }
    return None

In [ ]:
DAY_PLANNER_SYSTEM_PROMPT = """
You are a Personalized Day Planner AI assistant. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You call one function at a time to assist with the user's planning needs. Don't make assumptions about task values.
Pay special attention to the schema types and validation.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>, "arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools>
%s
</tools>

Example sessions:

1. Adding a new task:
<question>Add a task to prepare presentation for tomorrow at 2 PM with high priority</question>
<thought>I need to add a new task with priority 1 (highest) for tomorrow's presentation</thought>
<tool_call>
{
    "name": "add_task",
    "arguments": {
        "task": "Prepare presentation",
        "priority": 1,
        "time": "14:00",
        "date": "2024-11-09",
        "deadline": "2024-11-09"
    },
    "id": 0
}
</tool_call>

<observation>{"status": "success", "message": "Task created successfully", "task_id": "507f1f77bcf86cd799439011"}</observation>
<response>I've added your high-priority presentation task for tomorrow at 2 PM. The task has been successfully created.</response>

2. Retrieving tasks:
<question>What are my tasks for tomorrow?</question>
<thought>I need to fetch all tasks scheduled for tomorrow</thought>
<tool_call>
{
    "name": "display_tasks",
    "arguments": {
        "date": "2024-11-09"
    },
    "id": 1
}
</tool_call>

<observation>
{
    "tasks": [
        {
            "task_id": "507f1f77bcf86cd799439011",
            "task": "Prepare presentation",
            "priority": 1,
            "time": "14:00",
            "date": "2024-11-09",
            "deadline": "2024-11-09"
        }
    ]
}
</observation>
<response>For tomorrow (November 9, 2024), you have one high-priority task:
- Prepare presentation at 2:00 PM</response>

Additional constraints:

- Priority levels must be between 1-5 (1 being highest priority)
- Time must be in 24-hour format (HH:MM)
- Dates must be in YYYY-MM-DD format
- All database operations must be acknowledged with appropriate success/error messages
- If a task creation fails, provide specific error details and suggest corrections
- For updates and deletions, verify the task_id exists before attempting the operation
- If the user asks something unrelated to task management, answer freely using <response></response> tags
- When displaying tasks, sort them by time and priority
"""

In [ ]:
# Existing functions remain the same
@tool
def add_task(task: str, time: str, date: str, deadline: str = None,priority: int=3,):
    if(date=="tomorrow"):
        date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    if(date=="today"):
        date = datetime.now().strftime('%Y-%m-%d')
    if(date=="day after tomorrow"):
        date = (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d')
    if(time=="now"):
        time = datetime.now().strftime('%H:%M')
    if(deadline=="today"):
        deadline = datetime.now().strftime('%Y-%m-%d')
    if(deadline=="tomorrow"):
        deadline = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    
    task_data = {
        "task": task,
        "priority": priority,
        "time": time,
        "date": date,
        "deadline": deadline
    }
    tasks_collection.insert_one(task_data)
    return "Task added successfully."
@tool
def delete_task(task: str, date: str,priority: int=0, time: str=None, deadline: str = None):
    if(date=="tomorrow"):
        date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    if(date=="today"):
        date = datetime.now().strftime('%Y-%m-%d')
    if(date=="day after tomorrow"):
        date = (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d')
    if(time=="now"):
        time = datetime.now().strftime('%H:%M')
    if(deadline=="today"):
        deadline = datetime.now().strftime('%Y-%m-%d')
    if(deadline=="tomorrow"):
        deadline = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    
    tasks_collection.delete_one({
            "task": task,
            "date": date
        })
    return "Task deleted successfully."
    




@tool
def display_tasks(date: str):
    if(date=="tomorrow"):
        date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    if(date=="today"):
        date = datetime.now().strftime('%Y-%m-%d')
    if(date=="day after tomorrow"):
        date = (datetime.now() + timedelta(days=2)).strftime('%Y-%m-%d')
    tasks = tasks_collection.find({"date": date}).sort("priority", 1)
    return list(tasks)



available_tools = {
    "add_task": add_task,
    "delete_task": delete_task,
    "display_tasks": display_tasks,
    
}


In [ ]:
tools_signature = add_task.fn_signature + ",\n" + delete_task.fn_signature + ",\n" + display_tasks.fn_signature 

In [ ]:
#print(tools_signature)

In [ ]:
#DAY_PLANNER_SYSTEM_PROMPT =DAY_PLANNER_SYSTEM_PROMPT + tools_signature
#print(DAY_PLANNER_SYSTEM_PROMPT )

In [ ]:
class DayPlannerAgent(ReactAgent):
    def __init__(self, tools):
        super().__init__(tools)
    
    def process_input(self, user_msg: str) -> Dict[str, Any]:
        """Process natural language input and extract relevant parameters."""
        datetime_info = parse_datetime(user_msg)
        if not datetime_info:
            return {"error": "Could not extract date and time from the message"}
        
        # Extract meeting description/title
        # Remove date/time patterns first
        clean_msg = re.sub(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', '', user_msg)
        clean_msg = re.sub(r'\d{1,2}:\d{2}(?:\s*(?:am|pm)?)', '', clean_msg, flags=re.IGNORECASE)
        
        # Look for meeting subject after common phrases
        subject_patterns = [
            r'(?:discuss|about|regarding|for|on)\s+(.+?)(?:\.|$)',
            r'meeting\s+(?:for|about)?\s+(.+?)(?:\.|$)',
        ]
        
        meeting_subject = None
        for pattern in subject_patterns:
            match = re.search(pattern, clean_msg, re.IGNORECASE)
            if match:
                meeting_subject = match.group(1).strip()
                break
        
        if not meeting_subject:
            meeting_subject = "Untitled Meeting"
        
        return {
            "date": datetime_info['date'],
            "time": datetime_info['time'],
            "meeting_name": meeting_subject
        }
    
    def run(self, user_msg: str):
        """Process the user message and execute appropriate actions."""
        params = self.process_input(user_msg)
        if "error" in params:
            return params["error"]
        
        return schedule_meeting(
            meeting_name=params["meeting_name"],
            time=params["time"],
            date=params["date"]
        )

In [ ]:
USER_QUESTION="Hi, How can you help me?"
chat_history = [
    {
        "role": "system",
        "content": DAY_PLANNER_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [ ]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

In [ ]:
agent = ReactAgent(tools=[
        add_task,
        delete_task,
        display_tasks,
    ]) 

In [ ]:
#agent.run("Create a task for tomorrow to prepare a presentation at 9 am")

In [ ]:
#agent.run("What are all the tasks for tomorrow?")

In [ ]:
#agent.run("Delete tomorrow's task to prepare presentation")

In [ ]:

import gradio as gr

# Define the agent_response function
def agent_response(user_input):
    return agent.run(user_input)

# Create and launch the Gradio interface
description = """
# Your AI-powered personal assistant for managing tasks and meetings

You can:
- Add tasks or meetings
- View your schedule
- Delete items

Just type your request in natural language!
"""

iface = gr.Interface(
    fn=agent_response,
    inputs=gr.Textbox(placeholder="Type your request here..."),
    outputs=gr.Textbox(lines=10, label="Assistant Response"),  # Increased size with 10 lines
    description=description,
    title="AI Task Assistant"
)
iface.launch()
